In [1]:
import pandas as pd
import numpy as np
import os
import pyarrow
from pyarrow import csv

In [5]:
#df = pd.read_csv(r"C:\Users\Valerie\Downloads\Rew_Neutral.csv", header = None)
df = pd.read_csv(r"C:\Users\Valerie\Downloads\Rew_Neutral_Pun_HEY1.csv", header = None)
trials = df.iloc[:,1]
#df2 = pd.read_excel(r"C:\Users\Valerie\Downloads\og_t_i_p.xlsx")
rewardtrials = []
ititrials = []
puntrials = []
for i in range(0, len(df.iloc[:,1])):
    if (df.iloc[:,1][i] == 'p'):
        puntrials.append(i)
    elif (df.iloc[:,1][i] == 'i'):
        ititrials.append(i)
    elif (df.iloc[:,1][i] == 't'):
        rewardtrials.append(i)

# BRUTE FORCE WAY

Please define (run) `process_one_folder` before running `process_all_folders`

In [3]:
def remove_consecutive(input_list, limit=5):
    result = [input_list[0]] 

    for value in input_list[1:]:
        if abs(value - result[-1]) > limit:
            result.append(value)

    return result

def process_one_folder(folder, method):
    dcol = ['D' + str(i) for i in range(128)]
    
    if method == 'guppy':
        for file in os.listdir(folder):
            cur_csv = os.path.join(folder,file)
            df = pd.read_csv(cur_csv)

            if file.endswith('TTL.csv'):

                df['ttls'] = df[dcol].max(axis=1)

                newdf = pd.DataFrame()
                t = list(df['TIME'].loc[df['ttls'] == 1])
                newdf['timestamps'] = remove_consecutive(t, limit=8.5)
                newdf = newdf.reset_index(drop=True)

                newdf.to_csv(os.path.join(folder,'TTL_guppy_processed_2.csv'), index=None)
            elif file.endswith('405.csv') or file.endswith('470.csv'):
                hz =  1 / df['Sampling_Freq'][0] 
                newdf = pd.DataFrame({'data': df[dcol].to_numpy().flatten(), 'sampling_rate': df['Sampling_Freq'][0]})
                newdf['timestamps'] = newdf.index * hz
                newdf['sampling_rate'][1:] = None
                newdf = newdf[['timestamps', 'data' , 'sampling_rate']]
                newdf_ = pyarrow.Table.from_pandas(newdf)
                pyarrow.csv.write_csv(newdf_, os.path.join(folder, file.split('.')[0] + '_guppy_processed.csv') )
    
    elif method == 'pmat':
        df_405 = pd.read_csv(os.path.join(folder,'405.csv'))
        df_470 =  pd.read_csv(os.path.join(folder,'470.csv'))

        newdf = pd.DataFrame()
        newdf['TimeStamp'] = df_470['TIME'].copy()
        newdf['Signal'] = df_470[dcol].mean(axis=1)
        newdf['Control'] = df_405[dcol].mean(axis=1)

        newdf.to_csv(os.path.join(folder,'pmat_processed.csv'))
    
    elif method == 'pmatttl':
        df_ttl = pd.read_csv(os.path.join(folder,'TTL.csv'))
        df_405 = pd.read_csv(os.path.join(folder,'405.csv'))
        df_470 =  pd.read_csv(os.path.join(folder,'470.csv'))

        newdf = pd.DataFrame()
        newdf['TimeStamp'] = df_470['TIME'].copy()
        newdf['Signal'] = df_470[dcol].mean(axis=1)
        newdf['Control'] = df_405[dcol].mean(axis=1)
        newdf['ttls'] = df_ttl[dcol].max(axis=1)

        firstidx = np.where(newdf.ttls == 1)[0][0]

        newdf['TimeStamp'] -= newdf['TimeStamp'][firstidx]
        newdf = newdf[['TimeStamp', 'Signal', 'Control']][firstidx:].reset_index(drop=True)

        newdf.to_csv(os.path.join(folder,'pmat_ttl_processed.csv'))

In [4]:
def process_all_folders(folders, method):
    
    for root, directories, files in os.walk(folders):
        for directory in directories:
            directory_path = os.path.join(root, directory)
            
            # if all three files exists
            if all(item in os.listdir(directory_path) for item in ('405.csv', '470.csv', 'TTL.csv')):
                process_one_folder(directory_path, method)
            
            print(f'Processed: {directory_path} with {method}')

In [112]:
%%time
process_one_folder(r'C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1018_cohort\test\rew_neutral_punish\RNP_D3\99_3', 'guppy')

CPU times: total: 4.58 s
Wall time: 4.46 s


In [5]:
%%time
folder = r"C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass"

process_all_folders(folder, 'guppy')

Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD1_umass with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD2_umass with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD3_umass with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD1_umass\47_1 with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD1_umass\47_3 with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD1_umass\48_1 with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD1_umass\48_2 with guppy
Processed: C:\Users\Valerie\Documents\Tejeda_Lab\headfixed\1130_umass\Raw_Photom_Data\RTD2_umass\47_1 with guppy
Processed: C:\